Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3_2.json")


In [3]:
def recuperate_datasets(wf_char_p: float, wf_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)

  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        corpus_1 = 'wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the evaluation function ⚙️

Let us initialize the evaluation object.

In [4]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [5]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5

In [6]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.3782899747775979,
    'fr_word_p': 0.010927808357027502,
    'learning_rate': 0.002147597238420693,
    'weight_decay': 0.03719050938512442,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 977,
    'epochs': 200,
    'mid_epoch': 488,
    'max_len': 22,
    'end_mark': 4,
    'bleu': 5.4679,
    'model_dir': 'data/checkpoints/wf_t5_small_custom_train_v5_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_wf_v5/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 5, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_wf"
                )

# We will train from the model from the checkpoints 
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### --- Wandb v5_2

486

In [6]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.5843957796546101,
    'fr_word_p': 0.03941913506751615,
    'learning_rate': 0.008606183100102608,
    'weight_decay': 0.011552242096629329,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 681,
    'epochs': 600,
    'mid_epoch': 68,
    'max_len': 36,
    'end_mark': 4,
    'bleu': 9.6054,
    'model_dir': 'data/checkpoints/wf_t5_small_custom_train_v5_2_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_wf_v5_2/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name,
                                                       eos_token_id = tokenizer.eos_token_id,
                                                       pad_token_id = tokenizer.pad_token_id)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = '5_2', evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_wf"
                )

# We will train from the model from the checkpoints 
trainer.load(config['model_dir'], load_best=True) # Only for the first loading
# trainer.load(config['new_model_dir'], load_best=True)

        

### ---

In [7]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/63 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.008542350491475578]}


Test batch number 13: 100%|██████████| 13/13 [00:07<00:00,  1.79batches/s]



Metrics: {'train_loss': 0.38997814525859525, 'test_loss': 1.124950647354126, 'bleu': 8.8807, 'gen_len': 13.197}




  2%|▏         | 1/63 [01:22<1:25:43, 82.96s/it]

...

 98%|█████████▊| 62/63 [36:31<00:33, 33.54s/it]

For epoch 68: 
{Learning rate: [0.007750826144500393]}


Test batch number 13: 100%|██████████| 13/13 [00:07<00:00,  1.79batches/s]



Metrics: {'train_loss': 0.026897917219409435, 'test_loss': 1.5275334119796753, 'bleu': 17.0143, 'gen_len': 11.7828}




100%|██████████| 63/63 [37:07<00:00, 35.36s/it]


### Predictions and Evaluation

In [7]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [8]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 13: 100%|██████████| 13/13 [00:21<00:00,  1.62s/batches]


In [9]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
188,Joxal kenn liggéey bi!,Donne le travail à un autre!,Il n'y a personne qui puisse m'aider.
189,Góor gii demoon.,Cet homme qui a été.,L'homme qui avait l'habitude de sortir.
190,Nileen ka.,Dites-lui.,Dites-leur.
191,Ma ŋgoogule foofu.,Il est là.,Vous voilà là.
192,Gis ŋga nit kale?,Tu vois cet homme là-bas?,Tu as vu l'homme?
193,Ma defe góor gi dem na!,L'homme est parti je crois!,Est-ce l'homme qui est parti.
194,Séen naa ag guy.,J'ai aperçu un baobab.,J'ai aperçu une femme.
195,Kii dafa demkoon.,Celui-ci serait parti.,Celui-ci allait partir.
196,Bunt yi yépp a tëjju woon.,Toutes les portes étaient ouvertes.,Les travailleurs c'est nous.
197,"Faatim la, tay.","C'est Fatim, aujourd'hui.",C'est peut-être Fatim.


In [10]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
18,Ndax kan dem?,Afin que parte qui?,Afin que parte?
168,Gis ŋga sa yeneen xarit yi?,Tu as vu les autres amis?,Tu as vu quels amis à toi?
63,Jigéen ñan ñoo réer?,Quelles femmes se sont égarées?,Quelles personnes sont venues?
175,Li ŋga wax la ndaw si taamu.,C'est ce que tu as dit que la jeune femme préf...,Ce que tu as dit.
71,Man mii demuma.,Moi-même je n'ai pas été.,Moi je ne parte pas.
86,Dem nañu.,Ils partent.,Nous sommes partis.
5,Nit ñi daawuñu coow.,"Les gens n'étaient pas, habituellement, bavards.",Les gens ne sont pas si fameux.
118,Naka ngon si.,Bonne soirée.,Comment se passe la nuit.
12,Meneen xar mépp moo gis.,Tout autre mouton que tu vois.,Ce mouton n'est pas gras?
150,Nataalu juróom benn fukk ak juróom ñeent day w...,Une photo datant d'il y a soixante et neuf ans...,Le rayonnement peut également être contenu pen...
